# Bounding configuration GP example

This notebook is designed to test whether a simple simple comprised of bounding boxes can be generated using [deap](http://deap.readthedocs.io/en/master/api/tools.html), a Python Evolutionary Algorithm Package.

In [1]:
import copy

import numpy as np

from functools import partial
import types

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from IPython.display import Image

In [2]:
from OCC.gp import gp_Pnt
from OCC.gp import gp_Ax2, gp_Ax1, gp_Dir
from OCC.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere

from airconics.base import AirconicsShape, AirconicsCollection
from airconics.topology import Topology, LSURF_FUNCTIONS
from airconics.examples.wing_example_transonic_airliner import *
from airconics.examples.tailplane_example_transonic_airliner import *
from airconics.examples.straight_wing import *
from airconics import AirCONICStools as act

import pydot
import types
import functools
from collections import OrderedDict


# Create a simple Box class that inherits from AirconicsShape
#  - this will behave similarly to the Engine, Fuselage class etc.
class Box(AirconicsShape):
    def __init__(self, xmin, ymin, zmin, dx, dy, dz):
        # This implicitly calls build
        super(Box, self).__init__(construct_geometry=True,
                                             xmin=xmin,
                                             ymin=ymin,
                                             zmin=zmin,
                                             dx=dx,
                                             dy=dy,
                                             dz=dz)
        
        
        bbox_xmin, bbox_ymin, bbox_zmin, bbox_xmax, bbox_ymax, bbox_zmax = self.Extents()
        self.bbox_xmin = bbox_xmin
        self.bbox_ymin = bbox_ymin
        self.bbox_zmin = bbox_zmin
        self.bbox_xmax = bbox_xmax
        self.bbox_ymax = bbox_ymax
        self.bbox_zmax = bbox_zmax
        
        directions = []
        
    def Build(self):
        Xmin = gp_Pnt(self.xmin, self.ymin, self.zmin)
        Xmax = gp_Pnt(self.xmin + self.dx, self.ymin + self.dy, self.zmin + self.dz)
        self['Box'] = BRepPrimAPI_MakeBox(Xmin, Xmax).Shape()

# Create a simple Box class that inherits from AirconicsShape
#  - this will behave similarly to the Engine, Fuselage class etc.
class Sphere(AirconicsShape):
    def __init__(self, xcenter, ycenter, zcenter, d):
        # This implicitly calls build
        super(Sphere, self).__init__(construct_geometry=True,
                                             xcenter=xcenter,
                                             ycenter=ycenter,
                                             zcenter=zcenter,
                                             d=d)
        
        bbox_xmin, bbox_ymin, bbox_zmin, bbox_xmax, bbox_ymax, bbox_zmax = self.Extents()
        self.bbox_xmin = bbox_xmin
        self.bbox_ymin = bbox_ymin
        self.bbox_zmin = bbox_zmin
        self.bbox_xmax = bbox_xmax
        self.bbox_ymax = bbox_ymax
        self.bbox_zmax = bbox_zmax
        
    def Build(self):
        Xcenter = gp_Pnt(self.xcenter, self.ycenter, self.zcenter)
        self['Sphere'] = BRepPrimAPI_MakeSphere(Xcenter, self.d/2).Shape()

# Use a wrapper to convert boxN, sphereN etc. to another function that returns a callable
def wrap_shapeN(shapeN):
    @functools.wraps(shapeN)
    def wrapped_shapeN(*args, **kwargs):
        return partial(shapeN, *args)
    return wrapped_shapeN

# # @autodefine_functions
# class BoxLayout(AirconicsCollection):
#     # Define the max number of sub components that can be attached to a parent geometry

#     def __init__(self,
#                  ComponentTypes=[LiftingSurface, Fuselage],
#                  MaxAttachments=2,
#                  ArgTypes={Box: [float] * 6,
#                            Sphere: [float] * 4,
#                            Fuselage: [float] * 7,
#                            LiftingSurface: [float] * 4}):

#         super(BoxLayout, self).__init__(parts={},
#                                         ComponentTypes=ComponentTypes,
#                                         MaxAttachments=2,
#                                         ArgTypes=ArgTypes,
#                                         construct_geometry=True,
#                                         mirror=False
#                                        )
        
#         # Carry around the tree for visualisation purposes:
#         self._Tree = []
#         self._deap_Tree = None   # This will be populated when 
        
#         # Start with a simple box
#         self.nparts = 0
#         self.routine = None 
        
#         # This will allow components to track what they should be attached to
#         # (Needs to be ordered so that the final entries can be removed)
#         self.parent_nodes = OrderedDict()
    
#     def __setitem__(self, name, part_w_arity):
#         """Overloads the assignment operator used by AirconicsCollection
#         to allow only tuples as inputs - arity must be specified for
#         topology.

#         Parameters
#         ----------
#         name - string
#         part_w_arity - tuple
#             (Airconics class, int), eg: (Fuselage, 2) is a Fuselage shape with
#             2 descendents in its topological tree

#         Notes
#         -----
#         appends to the self.Tree and self._OrderedParts attributes
#         """
#         try:
#             part, arity = part_w_arity
#         except:
#             print("Warning: no arity set. Treating as zero")
#             part = part_w_arity
#             arity = 0

#         node = TreeNode(part, name, arity)

#         self._Tree.append(node)
        
#         if len(self.parent_nodes) > 0:
#             if self.parent_nodes.values()[-1] > 1:
#                 self.parent_nodes[self.parent_nodes.keys()[-1]] -= 1
#             else:
#                 while self.parent_nodes.values()[-1] < 1:
#                     self.parent_nodes.popitem()
        
#         self.parent_nodes[name] = arity
                
        
#         super(BoxLayout, self).__setitem__(name, part)
    
#     def run(self, tree, pset):
#         self._reset()
#         routine = gp.compile(tree, pset)
#         self._deap_Tree = tree
#         routine()

#     def _reset(self):
#         self._Parts = {}
#         self._Tree = []
#         self._deap_Tree = None
#         self.parent_nodes.clear()
#         self.nparts = 0
    
#     def fit_to_parent(self, oldscaling, oldposition):
#         """Given an old scaling and old position (this will be a random number between 0 and 1
#         using the DEAP tree I have set up elsewhere), a new scaling and position is returned
#         allowing the resulting shape to 'fit' to its parent"""
#         # Need to get a scaling from the parent of arbitrary type:
#         # using a try-except to work for any parent type ... could probably do better here
#         parent = self[self.parent_nodes.keys()[-1]]
#         scalingrange = (0.6, 1)
        
#         # Ensure that the oldscaled and oldposition fractions does give something invisibly small:
#         oldscaling = np.interp(oldscaling, [0, 1], [0.6, 1])
#         try:
#             # The LiftingSurface branch (could probably do better here)
#             parentscalefactor = parent.ScaleFactor
            
#         except AttributeError:
#             # The Fuselage branch
#             parentscalefactor = parent.Scaling[0]
        
#         try:
#             # The LiftingSurface branch (could probably do better here)
#             parent_x = parent.ApexPoint.X()
#             xlength = parent.ScaleFactor * parent.ChordFactor
            
#         except AttributeError:
#             # The Fuselage branch
#             parent_x = parent.NoseCoordinates[0]
#             xlength = (parent.SternPoint.X() - parent.BowPoint.X()) * (parentscalefactor / 55.902)
#         # The scaling is some percentage of parent (assumes components get smaller)
#         newscaling = oldscaling * parentscalefactor
        
#         newx = parent_x + xlength * oldposition
        
#         return newscaling, newx
    
# #     def update_nodes(self):
# #         if self.nodes.values()[-1] > 1:
# #             self.nodes[self.nodes.keys()[-1]] -= 1
# #         elif self.nodes.values()[-1] == 1:
# #             self.nodes.popitem()
# #         else:
# #             pass
    
# #     @wrap_shapeN
# #     def useless_None(self):
# #         """"""
# #         self.
# #         return None
    
#     @wrap_shapeN
#     def mirror_subcomponentsN(self, *args):
#         self.mirror = True
#         for arg in args:
#             arg()
        
#         self.mirror = False

# #     @wrap_shapeN
# #     def boxN(self, xmin, ymin, zmin, dx, dy, dz, *args):
# #         # Fits N new components to this box layout
# #         box = Box(xmin, ymin, zmin, dx, dy, dz)
# #         # Do no be confused between the numbering of boxes and the number of descendent
# #         #  nodes: Each box needs a unique ID, so naming a box0 function "box0" replaces
# #         #  other shapes in this layout that are also named box0
# #         self['box{}_{}'.format(len(args), len(self))] = box, len(args)
        
# #         if self.mirror:
# #             super(BoxLayout, self).__setitem__('box{}_{}_mirror'.format(len(args), len(self)), box.MirrorComponents(plane='xz'))
        
# #         for arg in args:
# #             arg()
    
# #     @wrap_shapeN
# #     def sphereN(self, xcenter, ycenter, zcenter, r, *args):
# #         # Fits N new components to this box layout
# #         sphere = Sphere(xcenter, ycenter, zcenter, r)
# #         # Do no be confused between the numbering of boxes and the number of descendent
# #         #  nodes: Each box needs a unique ID, so naming a box0 function "box0" replaces
# #         #  other shapes in this layout that are also named box0
# #         self['sphere{}_{}'.format(len(args), len(self))] = sphere, len(args)
        
# #         for arg in args:
# #             arg()
    
#     @wrap_shapeN
#     def fuselageN(self, NoseLengthRatio, TailLengthRatio, ScalingX, FinenessRatio,
#                   NoseX, NoseY, NoseZ, *args):
#         """Parameter descriptions can be found from the airconics.Fuselage class, all are floats"""
#         # Need to add constraints here so that the Fuselage has a good chance of
#         # successful lofting: any out of range values will be moved to the
#         # nearest boundary:
#         arglimits = {NoseLengthRatio: (0.18, 0.19), TailLengthRatio: (0.29, 0.295),
#                     ScalingX: (0.5, 5), FinenessRatio:(0.5, 2)}
        
# #         NoseLengthRatio = np.interp(NoseLengthRatio, [0,1], arglimits[NoseLengthRatio])
# #         TailLengthRatio = np.interp(TailLengthRatio, [0,1], arglimits[TailLengthRatio])

#         # For now, I'm fixing these values to avoid errors (curve projection seems to fail)
#         NoseLengthRatio = 0.182
#         TailLengthRatio = 0.293

#         # Essentially this checks if the current shape is being fitted to a parent
#         if len(self.parent_nodes) > 0:
#             ScalingX, NoseX = self.fit_to_parent(ScalingX, NoseX)
#         else:
#              ScalingX = np.interp(ScalingX, [0,1], arglimits[ScalingX])
#         print(ScalingX, NoseX)

#         FinenessRatio = np.interp(FinenessRatio, [0, 1], arglimits[FinenessRatio])

#         ScalingYZ = ScalingX / FinenessRatio

#         NoseY = NoseZ = 0
        
# #         print(NoseLengthRatio, TailLengthRatio, Scaling)
#         # Fits N new components to this box layout
#         fus = Fuselage(NoseLengthRatio=NoseLengthRatio,
#                        TailLengthRatio=TailLengthRatio,
#                        Scaling=[ScalingX, ScalingYZ, ScalingYZ],
#                        NoseCoordinates=[NoseX, NoseY, NoseZ],
# #                        SimplificationReqd=True
#                       )
#         # Do no be confused between the numbering of boxes and the number of descendent
#         #  nodes: Each box needs a unique ID, so naming a box0 function "box0" replaces
#         #  other shapes in this layout that are also named box0
#         name = 'fuselage{}_{}'.format(len(args), len(self))
#         self[name] = fus, len(args)
        
#         if self.mirror:
#             super(BoxLayout, self).__setitem__(name+'_mirror', fus.MirrorComponents(plane='xz'))
                
#         for arg in args:
#             arg()
    
#     @wrap_shapeN
#     def liftingsurfaceN(self, ApexX, ApexY, ApexZ, ScaleFactor, *args):
        
# #         ScaleFactor = np.interp(ScaleFactor, [0,1], [1,50])
#         ChordFactor=1

#         # Class definition
#         NSeg = 10
        
#         Functionals_dict = {#'AirlinerWing': 
# #                                 {'SweepFunct':mySweepAngleFunctionAirliner,
# #                                  'DihedralFunct': myDihedralFunctionAirliner,
# #                                  'ChordFunct': myChordFunctionAirliner,
# #                                  'TwistFunct': myTwistFunctionAirliner,
# #                                  'AirfoilFunct': myAirfoilFunctionAirliner
# #                                 },
#                             'AirlinerTP':
#                                 {'SweepFunct':mySweepAngleFunctionTP,
#                                  'DihedralFunct': myDihedralFunctionTP,
#                                  'ChordFunct': myChordFunctionTP,
#                                  'TwistFunct': myTwistFunctionTP,
#                                  'AirfoilFunct': myAirfoilFunctionTP
#                                 },
#                             'AirlinerFin':
#                                 {'SweepFunct': mySweepAngleFunctionFin,
#                                  'DihedralFunct': myDihedralFunctionFin,
#                                  'ChordFunct': myChordFunctionFin,
#                                  'TwistFunct': myTwistFunctionFin,
#                                  'AirfoilFunct': myAirfoilFunctionFin
#                                 },
# #                             'StraightWing': 
# #                                 {'SweepFunct':SimpleSweepFunction,
# #                                  'DihedralFunct': SimpleDihedralFunction,
# #                                  'ChordFunct': SimpleChordFunction,
# #                                  'TwistFunct': SimpleTwistFunction,
# #                                  'AirfoilFunct': SimpleAirfoilFunction
# #                                 }
#                            }
        
#         surfacetype = np.random.choice(Functionals_dict.keys())
        
#         # Essentially this checks if the current shape is being fitted to a parent
#         if len(self.parent_nodes) > 0:
#             ScaleFactor, ApexX = self.fit_to_parent(ScaleFactor, ApexX)
#             print(ScaleFactor, ApexX)
#         else:
#             ApexX = 0
                
#         ApexZ = ApexY = 0
#         P = (ApexX, ApexY, ApexZ)
#         apex = gp_Pnt(*P)
        
#         # Instantiate the class
#         wing = liftingsurface.LiftingSurface(P,
#                                              SegmentNo=NSeg,
#                                              ScaleFactor=ScaleFactor,
#                                              ChordFactor=ChordFactor,
#                                              **Functionals_dict[surfacetype])

#         # Rotate the component if necessary:
#         if surfacetype in ['AirlinerFin', 'StraightWing']:
#             rotation = np.random.choice([0, 32.5])#, 90]) # V tail or vertical fin
#             RotAx = gp_Ax1(gp_Pnt(*P), gp_Dir(1, 0, 0))
#             wing.RotateComponents(RotAx, rotation)

#         self['liftingsurface{}_{}'.format(len(args), len(self))] = wing, len(args)

#         if self.mirror:
#             super(BoxLayout, self).__setitem__('liftingsurface{}_{}_mirror'.format(len(args), len(self)-1), wing.MirrorComponents(plane='xz'))
        
#         for arg in args:
#             arg()
    
#     def temp_fitness(self):
#         """Until I come with something useful, the fitness evaluation for this
#         configuration will be based on the bounding box volume"""
#         try:
#             xmin, ymin, zmin, xmax, ymax, zmax = self.Extents()
#         except:
#             # Bounding Box was probably void
#             return 0
#         return (xmax-xmin) * (ymax-ymin)*(zmax-zmin)
    
#     def pydot_graph(self):
#         """Returns a pydot graph instance of a human readable interpretation of
#         the topology tree"""
#         # Use the _deap_Tree if one is available (stored on calling run(tree, pset)):
#         if self._deap_Tree:
#             # Note: ns below is a simple range list for every edge/label
#             nodes, edges, labels = gp.graph(self._deap_Tree)
#         else:
#             raise AttributeError("No DEAP GP tree was found: see Box_Layout.run(tree, pset)")
        
#         mirror_flag = False
        
#         graph = pydot.Dot(splines='ortho', ranksep='0.1')
#         graph.set_node_defaults(style='filled')
#         graph.set_edge_defaults(arrowhead='none')

#         cluster_1 = pydot.Cluster('standard', color='invis')
#         graph.add_subgraph(cluster_1)
        
#         # Check if a mirror plane exists in the geometry (affects tree visualisation)
#         if any(isinstance(component, gp_Ax2) for component in self.values()):
#             cluster_2 = pydot.Cluster('mirrored', label='mirrored', style='dashed')
#             graph.add_subgraph(cluster_2)
        
#         # i is used to increment the number of Geometric parts that have been found in
#         # the main loop (required as component names are prepended with their ordinal component
#         # number, and component names need to be matched with the _deap_tree)
#         i = 0
        
#         for node in nodes:
#             # note: this only works because the node number is manually added
#             # to the label by boxn:
# #             print(node)
#             try:
#                 label = "{}_{}".format(labels[node], i)
# #                 print(label)
#                 node_type = type(self[label])
                
#                 if isinstance(self[label], gp_Ax2):
#                     # Everything below here is mirrored, so switch on mirror_flag
#                     mirror_flag = True
#                     # Also retrieve the index of the mirror, as edges will be removed
#                     i_mirror = node
                
#                 i += 1

#             except KeyError:
#                 # If we get here, the node is not an AirconicsShape: get the return type
#                 # using the deap tree
#                 node_type = self._deap_Tree[node].ret
#                 try:
#                     label = "{:.4f}".format(labels[node])
#                 except ValueError:
#                     # if the formatting failed, it's probably a function name (str)
#                     label = labels[node]
                
#             pydot_node = pydot.Node(node, label=label, **NODE_PROPERTIES[node_type])
            
#             if mirror_flag:
#                 cluster_2.add_node(pydot_node)
            
#             else:
#                 cluster_1.add_node(pydot_node)
    
#         # Remove all (src, dest) edge from the list of edges if one of them
#         # points to/from the mirror node
#         if mirror_flag:
#             edges = [edge for edge in edges if i_mirror not in edge]
        
#         for edge in edges:
#             src, dest = edge
#             pydot_edge = pydot.Edge(src, dest)
#             graph.add_edge(pydot_edge)

#         return graph


In [3]:
config = Topology()


# pset = gp.PrimitiveSetTyped("MAIN", [], nt)

# def add_automatic_primitives(layout, pset):
#     """Adds the automatically defined primitives to the primitive set"""
#     for comptype in layout.ComponentTypes:
#         for i in range(layout.MaxAttachments + 1):
#             name = comptype.__name__.lower()
#             # get Number of inputs of the basic method e.g. fuselageN, and add
#             # This many (-1 due to self) float arguments to the typed primitive
#             argtypes = layout.ArgTypes[comptype] + [types.NoneType] * i
#             pset.addPrimitive(getattr(layout, name+'N'), argtypes, types.NoneType, name=name+str(i))
#     return None

# # pset.addPrimitive(config.box0, [float, float, float, float, float, float], nt, name='box0')
# # pset.addPrimitive(config.box1, [float, float, float, float, float, float, nt], nt, name='box1')
# # pset.addPrimitive(config.box2, [float, float, float, float, float, float, nt, nt], nt, name='box2')
                                              
# add_automatic_primitives(config, pset)
                                              
# pset.addPrimitive(np.random.rand, [], float)


# # def useless_None():
# #     return None
# # pset.addTerminal(useless_None, nt)

# # Mirroring primitives:
# def add_mirror_primitives(layout, pset):
#     for i in range(layout.MaxAttachments + 1):
#         name = 'mirror' + str(i)

#         pset.addPrimitive(layout.mirror_subcomponentsN, [types.NoneType] * i, types.NoneType, name=name)

# # def get_random_polyfunction(order):
# #     """Maximum order 2"""
# #     coeffs = np.random.rand(order+1)
# #     return np.poly(coeffs)

# add_mirror_primitives(config, pset)

# pset.addEphemeralConstant('rand', np.random.rand, float)


# creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

# toolbox = base.Toolbox()

# # Attribute generator
# toolbox.register("expr_init", gp.genFull, pset=pset, min_=2, max_=4)

# # Structure initializers
# toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
# toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [4]:
# Try to compile a single individual and visualise both the tree and geometry
config.randomize()

print(config)

from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()

config.Display(renderer)

renderer

liftingsurface1(rand(), rand(), rand(), rand(), rand(), random_lsurfdict(), liftingsurface2(rand(), rand(), rand(), rand(), rand(), random_lsurfdict(), fuselage0(rand(), rand(), rand(), rand(), rand(), rand(), rand()), mirror2(fuselage1(0.3705747076737227, 0.46398280246604084, 0.7022771263583513, 0.49785400537329094, 0.7871242319815318, 0.1169269150777068, 0.9266273390305896, empty), liftingsurface0(0.984714899197457, 0.8900378237922032, 0.48831972968220183, 0.844343605491652, 0.6601167673065461, AirlinerTP)), mirror1(liftingsurface2(0.6028221669981605, 0.029607827208340654, 0.8948466470446158, 0.773474552160385, 0.90437051406448, AirlinerWing, empty, empty, empty))))


<TornadoWebRenderer at http://PChambers_ngcm:38178/get/ddb46e478f2a4985a5eb7e6774d27a6d>

In [5]:
print(config)

liftingsurface1(rand(), rand(), rand(), rand(), rand(), random_lsurfdict(), liftingsurface2(rand(), rand(), rand(), rand(), rand(), random_lsurfdict(), fuselage0(rand(), rand(), rand(), rand(), rand(), rand(), rand()), mirror2(fuselage1(0.3705747076737227, 0.46398280246604084, 0.7022771263583513, 0.49785400537329094, 0.7871242319815318, 0.1169269150777068, 0.9266273390305896, empty), liftingsurface0(0.984714899197457, 0.8900378237922032, 0.48831972968220183, 0.844343605491652, 0.6601167673065461, AirlinerTP)), mirror1(liftingsurface2(0.6028221669981605, 0.029607827208340654, 0.8948466470446158, 0.773474552160385, 0.90437051406448, AirlinerWing, empty, empty, empty))))


In [8]:
config2 = Topology()

expr_string = """fuselage1(0., 0., 0., 1.0, 0.182,
0.293, 0.5022, mirror2(liftingsurface0(0., 0., 0., 0.38, 0., AirlinerTP),
liftingsurface0(0.8172, 0., 0., 0.3214, 32.5, AirlinerFin)))"""

config2.from_string(expr_string)

print(config2)

from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()

config2.Display(renderer)

renderer



fuselage1(0.0, 0.0, 0.0, 1.0, 0.182, 0.293, 0.5022, mirror2(liftingsurface0(0.0, 0.0, 0.0, 0.38, 0.0, AirlinerTP), liftingsurface0(0.8172, 0.0, 0.0, 0.3214, 32.5, AirlinerFin)))


<TornadoWebRenderer at http://PChambers_ngcm:38636/get/e9e605ef31694a6c98e864dec0b97554>

In [ ]:
# config2.Write('GP_output.step')
graph = config2.pydot_graph()
Image(graph.create_png())


In [ ]:
# For now, I'll have to do this by hand...
graph = """digraph G {
ranksep="0.1";
nodesep="0.1";
# splines=ortho;
node [style=filled];
edge [arrowhead=none];
subgraph cluster_standard {
color=invis;
0 [shape=ellipse, fillcolor="#136ed4", label=fuselage1, fontcolor="white"];
1 [shape=ellipse, label=<X<SUB>0</SUB>>];
2 [shape=ellipse, label=<Y<SUB>0</SUB>>];
3 [shape=ellipse, label=<Z<SUB>0</SUB>>];
4 [shape=ellipse, label=<SF>];
5 [shape=ellipse, label=<L<SUB>N</SUB>/L>];
6 [shape=ellipse, label=<L<SUB>T</SUB>/L>];
7 [shape=ellipse, label="L/D"];
8 [shape=box, fillcolor="#136ed4", label="mirror2", fontcolor="white"];
}

subgraph cluster_mirrored {
style=dashed;
9 [shape=box, fillcolor="#136ed4", label=liftingsurface0, fontcolor="white"];
10 [shape=ellipse, label=<<O>X</O><SUB>1</SUB>>];
11 [shape=ellipse, label=<<O>Y</O><SUB>1</SUB>>];
12 [shape=ellipse, label=<<O>Z</O><SUB>1</SUB>>];
13 [shape=ellipse, label=<<O>SF</O><SUB>1</SUB>>];
20 [shape=ellipse, label=<Shape_functs<SUB>1</SUB>>];

rank=same;
14 [shape=box, fillcolor="#136ed4", label=liftingsurface0, fontcolor="white"];
15 [shape=ellipse, label=<<O>X</O><SUB>2</SUB>>];
16 [shape=ellipse, label=<<O>Y</O><SUB>2</SUB>>];
17 [shape=ellipse, label=<<O>Z</O><SUB>2</SUB>>];
18 [shape=ellipse, label=<<O>SF</O><SUB>2</SUB>>];
21 [shape=ellipse, label=<Shape_functs<SUB>2</SUB>>];
}

0 -> 8;
8-> 9;
8 -> 14;

0 -> 1;
0 -> 2;
0 -> 3;
0 -> 4;
0 -> 5;
0 -> 6;
0 -> 7;
# 0 -> 9;
# 0 -> 14;
9 -> 10;
9 -> 11;
9 -> 12;
9 -> 13;
9-> 20;
14 -> 15;
14 -> 16;
14 -> 17;
14 -> 18;
14->21;
}
"""
dotgraph = pydot.graph_from_dot_data(graph)
dotgraph.write_svg('/home/pchambers/Documents/PhD/SABTalk_2017/images/tree1.svg')
Image(dotgraph.create_png())

In [ ]:
# dotgraph.write_pdf('/home/pchambers/Documents/PhD/Aviation2017/aviation_2017_paper/figure/gplayout_tree2.pdf')